In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 🤖 Automated Deep Researcher Agent (Gemini + SerpApi + CrewAI)

This notebook builds a **multi-agent deep research system** using:

- **Gemini** (via `GOOGLE_API_KEY`) as the main LLM  
- **SerpApi** (via `SERPAPI_API_KEY`) for web search  
- **CrewAI** for coordinating multiple agents  
- **crewai-tools** for search + web scraping  

The pipeline:

1. **Search Agent** → Finds high-quality sources using SerpApi  
2. **Scraper Agent** → Scrapes the main content from each URL  
3. **Analyzer Agent** → Extracts key insights and themes  
4. **Writer Agent** → Produces a well-structured research report in Markdown  

We’ll go step-by-step, just like a clean Kaggle notebook. 🚀

# 🟦 Step 0 – Create a new Kaggle Notebook

Go to https://www.kaggle.com/

Click “Code” → “New Notebook”.

It will open a blank notebook with one empty code cell.

# 🟦 Step 1 – Add your API keys as Kaggle Secrets

Before writing code, set up secrets:

In the right sidebar, click “Settings”.

Scroll to “Secrets” section.

Add two secrets:

Name: GOOGLE_API_KEY → Value: your Gemini API key

Name: SERPAPI_API_KEY → Value: your SerpApi API key

Make sure they are saved (you should see them listed).

Now we’re ready to code.

# 🟦Step 2 – Install required packages (Cell 1)

In the first code cell, paste this and run:

In [ ]:
import sys

!{sys.executable} -m pip install --upgrade \
    crewai \
    crewai-tools \
    langchain-google-genai \
    serpapi \
    beautifulsoup4 \
    requests

After it finishes:

(Optional but recommended) click “Runtime → Restart session”.

Then continue with the next steps.

# 🟦 Step 3 – Load API keys (Cell 2)

Add a new code cell and paste:

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    secrets = UserSecretsClient()
    
    # Load Google Gemini key
    GOOGLE_API_KEY = secrets.get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
    
    # Load SERPAPI key
    SERPAPI_API_KEY = secrets.get_secret("SERPAPI_API_KEY")
    os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY   # ✅ FIXED
    print("✅ SERPAPI_API_KEY setup complete.")

except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' and 'SERPAPI_API_KEY' to Kaggle secrets. Details: {e}"
    )


✅ Gemini API key setup complete.
✅ SERPAPI_API_KEY setup complete.


🟦 Step 4 – Configure Gemini LLM + tools and Define the four agents (Cell 3)

Add a new code cell and paste:

In [2]:
import os
from kaggle_secrets import UserSecretsClient

from crewai import Agent, LLM
from crewai_tools import SerpApiGoogleSearchTool, ScrapeWebsiteTool

# --- Secrets ---
secrets = UserSecretsClient()
os.environ["GOOGLE_API_KEY"] = secrets.get_secret("GOOGLE_API_KEY")
os.environ["SERPAPI_API_KEY"] = secrets.get_secret("SERPAPI_API_KEY")

# --- LLM (Gemini) ---
llm = LLM(model="gemini-2.5-flash")

# --- Tools ---
search_tool = SerpApiGoogleSearchTool()   # SerpApi search
website_scraper_tool = ScrapeWebsiteTool()  # 👈 HERE
print("✅ search_tool and website_scraper_tool ready.")

# --- Agents ---

search_agent = Agent(
    role="Web Search Strategist",
    goal="Accurately identify and retrieve the most relevant web pages for a given research query.",
    backstory=(
        "You are an expert at crafting precise search queries and sifting through search results "
        "to pinpoint valuable, authoritative sources. You prioritize diverse and recent information."
    ),
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
)

scraper_agent = Agent(
    role="Information Extractor",
    goal="Efficiently scrape and extract the main textual content from provided URLs.",
    backstory=(
        "You are skilled at navigating web pages and extracting their core content, ignoring boilerplate, "
        "advertisements, and irrelevant elements. You handle various website structures."
    ),
    llm=llm,
    tools=[website_scraper_tool],  # ✅ now defined
    verbose=True,
    allow_delegation=False,
)

analyzer_agent = Agent(
    role="Data Analyst & Synthesizer",
    goal=(
        "Thoroughly read, analyze, and synthesize extracted content to identify key insights, arguments, "
        "and supporting data points relevant to the research query. Identify main viewpoints and potential biases."
    ),
    backstory=(
        "You possess an exceptional ability to discern patterns, extract critical information, identify contradictions "
        "or biases within diverse texts, and connect disparate pieces of information into a cohesive understanding."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=True,
)

writer_agent = Agent(
    role="Report Composer",
    goal=(
        "Compile all analyzed insights into a comprehensive, well-structured, and easy-to-understand research report, "
        "addressing the initial query fully."
    ),
    backstory=(
        "You are a master storyteller and academic writer, capable of weaving complex information into a clear, "
        "concise, and compelling narrative. You ensure logical flow, proper formatting, and accurate representation "
        "of all findings."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

print("✅ Agents created.")


✅ search_tool and website_scraper_tool ready.
✅ Agents created.


# 🟦 Step 5 – Define the tasks (Cell 4)

Add a new code cell and paste:

In [3]:
from crewai import Task, Crew

# 🧪 You’ll pass the actual query later via crew.kickoff(inputs=...)
# e.g. research_query = "latest trends in agentic AI"

# Task for Search Agent
search_task = Task(
    description=(
        'You are the Web Search Strategist. Find 5–10 highly relevant and authoritative web pages or '
        'articles for the research query: "{query}". Prioritize diverse sources like academic papers, '
        'reputable news outlets, and expert blogs.'
    ),
    expected_output=(
        'A markdown-formatted list of URLs (max 10) that are most likely to contain in-depth information '
        'on the research query. Format:\n'
        '- [Title](URL) – 1–2 bullet points summarizing why this source is useful.'
    ),
    agent=search_agent,
)

# Task for Scraper Agent
scrape_task = Task(
    description=(
        'You are the Information Extractor. For each URL provided in the search results, scrape the main '
        'content and return a list of cleaned texts. Focus on article body / main content and ignore ads, '
        'navigation, and unrelated boilerplate.'
    ),
    expected_output=(
        'A markdown-formatted list where each item corresponds to one URL:\n'
        '- URL: <url>\n'
        '  - Content: """<cleaned main text>"""\n'
        'If a URL fails to scrape, include a note like: "Failed to scrape: <reason>".'
    ),
    agent=scraper_agent,
    context=[search_task],  # Uses output from search_task
)

# Task for Analyzer Agent
analyze_task = Task(
    description=(
        'You are the Data Analyst & Synthesizer. Read all the scraped content and extract key arguments, '
        'facts, statistics, and recurring themes relevant to the initial query: "{query}". Identify the main '
        'viewpoints, supporting evidence, and any conflicting information. Group similar ideas logically.'
    ),
    expected_output=(
        'A structured summary of key insights, arguments, and data points, clearly categorized into sections '
        'such as "Main Themes", "Key Arguments", "Important Data & Statistics", "Conflicting Viewpoints", '
        'and "Notable Biases or Limitations". Ensure it directly addresses the original research query.'
    ),
    agent=analyzer_agent,
    context=[scrape_task],  # Uses output from scrape_task
)

# Task for Writer Agent
write_report_task = Task(
    description=(
        'You are the Report Composer. Using the analyzed insights, compile a comprehensive, well-structured '
        'research report. The report should have:\n'
        '1. Introduction – set the context and state the purpose of the report for "{query}".\n'
        '2. Main Body – sections for each key theme or viewpoint identified by the Analyzer Agent.\n'
        '3. Conclusion – summarize the findings and offer final thoughts.\n'
        'Ensure the report flows logically, is easy to read, and fully addresses the original query: "{query}".'
    ),
    expected_output=(
        'A final, well-formatted research report in markdown format.\n'
        'Title: "{query} – Research Report".\n'
        'Use headings (##, ###) and bullet points where helpful.'
    ),
    agent=writer_agent,
    context=[analyze_task],  # Uses output from analyze_task
)

print("✅ Tasks created and wired to agents.")

# (Optional) put everything into a Crew and run it:
crew = Crew(
    agents=[search_agent, scraper_agent, analyzer_agent, writer_agent],
    tasks=[search_task, scrape_task, analyze_task, write_report_task],
    verbose=True,
)

# Example run:
# research_query = "latest trends in agentic AI"
# result = crew.kickoff(inputs={"query": research_query})
# print(result)


✅ Tasks created and wired to agents.


# 🟦 Step 7 – Create the Crew, run it, and save the report (Cell 6)

Add one more code cell and paste:

In [5]:
# ===========================================
#  Deep Researcher Multi-Agent (Gemini + SerpApi)
#  Complete Program
# ===========================================

import os
from kaggle_secrets import UserSecretsClient

from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerpApiGoogleSearchTool, ScrapeWebsiteTool

# ------------------------------
# 1. Load API keys from Kaggle
# ------------------------------
secrets = UserSecretsClient()

# Google Gemini key
GOOGLE_API_KEY = secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ GOOGLE_API_KEY (Gemini) setup complete.")

# SerpApi key
SERPAPI_API_KEY = secrets.get_secret("SERPAPI_API_KEY")
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY
print("✅ SERPAPI_API_KEY (SerpApi) setup complete.")

# ------------------------------
# 2. Configure LLM (Gemini via CrewAI)
# ------------------------------
llm = LLM(model="gemini-2.5-flash")
print("✅ Gemini LLM configured.")

# ------------------------------
# 3. Tools: SerpApi search + scraper
# ------------------------------
search_tool = SerpApiGoogleSearchTool()   # Uses SERPAPI_API_KEY
website_scraper_tool = ScrapeWebsiteTool()

print("✅ search_tool and website_scraper_tool ready.")

# ------------------------------
# 4. Agents
# ------------------------------

# Research Agent
search_agent = Agent(
    role="Web Search Strategist",
    goal="Accurately identify and retrieve the most relevant web pages for a given research query.",
    backstory=(
        "You are an expert at crafting precise search queries and sifting through search results "
        "to pinpoint valuable, authoritative sources. You prioritize diverse and recent information."
    ),
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
)

# Scraper Agent
scraper_agent = Agent(
    role="Information Extractor",
    goal="Efficiently scrape and extract the main textual content from provided URLs.",
    backstory=(
        "You are skilled at navigating web pages and extracting their core content, ignoring boilerplate, "
        "advertisements, and irrelevant elements. You handle various website structures."
    ),
    llm=llm,
    tools=[website_scraper_tool],
    verbose=True,
    allow_delegation=False,
)

# Analyzer Agent
analyzer_agent = Agent(
    role="Data Analyst & Synthesizer",
    goal=(
        "Thoroughly read, analyze, and synthesize extracted content to identify key insights, arguments, "
        "and supporting data points relevant to the research query. Identify main viewpoints and potential biases."
    ),
    backstory=(
        "You possess an exceptional ability to discern patterns, extract critical information, identify contradictions "
        "or biases within diverse texts, and connect disparate pieces of information into a cohesive understanding."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=True,
)

# Writer Agent
writer_agent = Agent(
    role="Report Composer",
    goal=(
        "Compile all analyzed insights into a comprehensive, well-structured, and easy-to-understand research report, "
        "addressing the initial query fully."
    ),
    backstory=(
        "You are a master storyteller and academic writer, capable of weaving complex information into a clear, "
        "concise, and compelling narrative. You ensure logical flow, proper formatting, and accurate representation "
        "of all findings."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

print("✅ Agents created.")

# ------------------------------
# 5. Tasks
# ------------------------------

# Search Task
search_task = Task(
    description=(
        'You are the Web Search Strategist. Find 5–10 highly relevant and authoritative web pages or '
        'articles for the research query: "{query}". Prioritize diverse sources like academic papers, '
        'reputable news outlets, and expert blogs.'
    ),
    expected_output=(
        'A markdown-formatted list of URLs (max 10) that are most likely to contain in-depth information '
        'on the research query. Format:\n'
        '- [Title](URL) – 1–2 bullet points summarizing why this source is useful.'
    ),
    agent=search_agent,
)

# Scrape Task
scrape_task = Task(
    description=(
        'You are the Information Extractor. For each URL provided in the search results, scrape the main '
        'content and return a list of cleaned texts. Focus on article body / main content and ignore ads, '
        'navigation, and unrelated boilerplate.'
    ),
    expected_output=(
        'A markdown-formatted list where each item corresponds to one URL:\n'
        '- URL: <url>\n'
        '  - Content: """<cleaned main text>"""\n'
        'If a URL fails to scrape, include a note like: "Failed to scrape: <reason>".'
    ),
    agent=scraper_agent,
    context=[search_task],
)

# Analyze Task
analyze_task = Task(
    description=(
        'You are the Data Analyst & Synthesizer. Read all the scraped content and extract key arguments, '
        'facts, statistics, and recurring themes relevant to the initial query: "{query}". Identify the main '
        'viewpoints, supporting evidence, and any conflicting information. Group similar ideas logically.'
    ),
    expected_output=(
        'A structured summary of key insights, arguments, and data points, clearly categorized into sections '
        'such as "Main Themes", "Key Arguments", "Important Data & Statistics", "Conflicting Viewpoints", '
        'and "Notable Biases or Limitations". Ensure it directly addresses the original research query.'
    ),
    agent=analyzer_agent,
    context=[scrape_task],
)

# Write Report Task
write_report_task = Task(
    description=(
        'You are the Report Composer. Using the analyzed insights, compile a comprehensive, well-structured '
        'research report. The report should have:\n'
        '1. Introduction – set the context and state the purpose of the report for "{query}".\n'
        '2. Main Body – sections for each key theme or viewpoint identified by the Analyzer Agent.\n'
        '3. Conclusion – summarize the findings and offer final thoughts.\n'
        'Ensure the report flows logically, is easy to read, and fully addresses the original query: "{query}".'
    ),
    expected_output=(
        'A final, well-formatted research report in markdown format.\n'
        'Title: "{query} – Research Report".\n'
        'Use headings (##, ###) and bullet points where helpful.'
    ),
    agent=writer_agent,
    context=[analyze_task],
)

print("✅ Tasks created.")

# ------------------------------
# 6. Crew
# ------------------------------
research_crew = Crew(
    agents=[search_agent, scraper_agent, analyzer_agent, writer_agent],
    tasks=[search_task, scrape_task, analyze_task, write_report_task],
    verbose=True,
    process=Process.sequential,   # tasks run in order
)

print("########################")
print("## Deep-Researcher Agent Initialized (Gemini) ##")
print("########################\n")

# ------------------------------
# 7. Run the crew
# ------------------------------
research_query = input("Enter your research query: ")

result = research_crew.kickoff(inputs={"query": research_query})

# Extract final text safely from CrewOutput
if hasattr(result, "output_text"):
    final_text = result.output_text
elif hasattr(result, "final_output"):
    final_text = result.final_output
else:
    final_text = str(result)

print("\n\n########################")
print("### FINAL RESEARCH REPORT ###")
print("########################\n")
print(final_text)

# ------------------------------
# 8. Save report to file
# ------------------------------
safe_name = (
    research_query.replace(" ", "_")
    .replace("/", "_")
    .replace(":", "_")
    .replace("?", "")
)[:50]

output_filename = f"{safe_name}_report_gemini.md"

with open(output_filename, "w", encoding="utf-8") as f:
    f.write(final_text)

print(f"\n📁 Report saved to {output_filename}")


✅ GOOGLE_API_KEY (Gemini) setup complete.
✅ SERPAPI_API_KEY (SerpApi) setup complete.
✅ Gemini LLM configured.
✅ search_tool and website_scraper_tool ready.
✅ Agents created.
✅ Tasks created.
########################
## Deep-Researcher Agent Initialized (Gemini) ##
########################



Enter your research query:  tell me about ms dhoni


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7e929f6a-d609-4c59-a0ca-d2ab0b855996                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Strategist                                                                                   │
│                                                                                                                 │
│  Task: You are the Web Search Strategist. Find 5–10 highly relevant and authoritative web pages or articles     │
│  for the research query: "tell me about ms dhoni". Prioritize diverse sources like academic papers, reputable   │
│  news outlets, and expert blogs.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: Google Gemini API error: 429 - You exceeded your current quota, please check your plan and billing      │
│  details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To    │
│  monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit.                                      │
│  * Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count,   │
│  limit: 250000, model: gemini-2.5-flash                                                                         │
│  Please retry in 51.544420286s.                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Information Extractor                                                                                   │
│                                                                                                                 │
│  Task: You are the Information Extractor. For each URL provided in the search results, scrape the main content  │
│  and return a list of cleaned texts. Focus on article body / main content and ignore ads, navigation, and       │
│  unrelated boilerplate.                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Information Extractor                                                                                   │
│                                                                                                                 │
│  Thought: Action: Read website content                                                                          │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

KeyboardInterrupt: 